In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
directory_path = '/content/drive/MyDrive/Newwave'
files = os.listdir(directory_path)
print(files)

Mounted at /content/drive
['ortho_normalized.tif', 'sample_rgb.tif', 'sample_rgb_reprojected.tif', 'full_sample_points.csv']


In [ ]:
# Import necessary libraries
import ee
import geemap
import folium
from google.colab import drive

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='nlcd-project')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


## Create Download link for Geotiff

In [ ]:
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Define the collection and point of interest
collection = ee.ImageCollection('USFS/GTAC/TreeMap/v2016')
point = ee.Geometry.Point(-85, 35.0)

# Define the region of interest (ROI) as a 10km x 10km buffer around the point
roi = point.buffer(10000).bounds()

# Select the canopy cover band (if available)
canopy_cover_band = 'STANDHT'  # Adjust this to the correct band name in the collection
image = collection.median().select(canopy_cover_band)

# Clip the image to the ROI
clipped_image = image.clip(roi)

# Generate a download URL
url = clipped_image.getDownloadURL({
    'scale': 30,  # Adjust scale (spatial resolution) as needed
    'region': roi.getInfo()['coordinates'],
    'fileFormat': 'GeoTIFF'
})

print("Download URL:")
print(url)


Download URL:
https://earthengine.googleapis.com/v1/projects/nlcd-project/thumbnails/1a1e37c011dad8cda9f76df2955500bd-3b0ab4425e03fa758043d6055ee38d16:getPixels


Select all three layers

In [ ]:
# Load the image collection
collection = ee.ImageCollection('USFS/GTAC/TreeMap/v2016')

# Mosaic the collection to combine all images into a single image
dataset = collection.mosaic()

# Select the desired layers
canopy_cover = dataset.select('CANOPYPCT')
live_trees_per_acre = dataset.select('TPA_LIVE')
stand_height = dataset.select('STANDHT')

# Combine the selected layers into a single image
selected_layers = dataset.select(['CANOPYPCT', 'TPA_LIVE', 'STANDHT'])

# Define a point of interest (Longitude, Latitude)
point = ee.Geometry.Point(-85, 35.0)  # Replace 37.0 with the desired latitude

# Define an area of interest (buffer around the point)
area_of_interest = point.buffer(10000)  # Buffer in meters (e.g., 10 km radius)

# Clip the combined image to the area of interest
layers_clipped = selected_layers.clip(area_of_interest)

# Get statistics for the selected layers in the area of interest
stats = layers_clipped.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=area_of_interest,
    scale=30  # Scale in meters (resolution of the dataset)
)

# Print the result
print('Layer statistics:', stats.getInfo())

# Export the clipped image to Google Drive (Optional)
task = ee.batch.Export.image.toDrive(
    image=layers_clipped,
    description='Selected_Layers_Export',
    folder='GEE_Exports',  # Google Drive folder name
    fileNamePrefix='selected_layers',
    scale=30,  # Scale in meters
    region=area_of_interest.bounds().getInfo()['coordinates'],
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    formatOptions={'cloudOptimized': True},
)

task.start()
print('Export task started. Check your Google Drive.')


Layer statistics: {'CANOPYPCT': 72.42239296636798, 'STANDHT': 66.14803321144628, 'TPA_LIVE': 705.1227720191678}
Export task started. Check your Google Drive.
